<a href="https://colab.research.google.com/github/Sarvesh1814/HateXplain/blob/Sarvesh/LSTM_Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
import spacy
import nltk
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from gensim.models import Word2Vec
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Reproducibility/Sample Model/HateXplain2.csv")

In [ ]:
df = df[['post_tokens',	'Target_cat',	'Label_cat',	'final_rationales']]
df['post_tokens'] = df['post_tokens'].apply(lambda x: eval(x))
df.head()

,post_tokens,Target_cat,Label_cat,final_rationales
0,"[i, dont, think, im, getting, my, baby, them, ...",None,normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[we, cannot, continue, calling, ourselves, fem...",None,normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[nawt, yall, niggers, ignoring, me]",African,normal,"[0, 0, 0, 0, 0]"
3,"[<user>, i, am, bit, confused, coz, chinese, p...",Asian,hatespeech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[this, bitch, in, whataburger, eating, a, burg...",Women,hatespeech,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
df['tweet']=''
for i in range(len(df)):
  df['tweet'].iloc[i] = " ".join(df["post_tokens"].iloc[i])

In [ ]:
df

,post_tokens,Target_cat,Label_cat,final_rationales,tweet
0,"[i, dont, think, im, getting, my, baby, them, ...",None,normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",i dont think im getting my baby them white 9 h...
1,"[we, cannot, continue, calling, ourselves, fem...",None,normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",we cannot continue calling ourselves feminists...
2,"[nawt, yall, niggers, ignoring, me]",African,normal,"[0, 0, 0, 0, 0]",nawt yall niggers ignoring me
3,"[<user>, i, am, bit, confused, coz, chinese, p...",Asian,hatespeech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",<user> i am bit confused coz chinese ppl can n...
4,"[this, bitch, in, whataburger, eating, a, burg...",Women,hatespeech,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",this bitch in whataburger eating a burger with...
...,...,...,...,...,...
20143,"[if, ur, still, on, twitter, tell, carlton, i,...",None,offensive,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...",if ur still on twitter tell carlton i said his...
20144,"[when, i, first, got, on, here, and, said, i, ...",African,offensive,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, ...",when i first got on here and said i hate trump...
20145,"[was, macht, der, moslem, wenn, der, zion, geg...",Other,normal,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",was macht der moslem wenn der zion gegen seine...
20146,"[it, is, awful, look, at, world, demographics,...",Asian,hatespeech,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, ...",it is awful look at world demographics asians ...


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Input, GlobalMaxPooling1D, Concatenate, Dot
from tensorflow.keras.models import Model
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import gensim.downloader as api

In [ ]:
model_name = 'glove-wiki-gigaword-300'
w2v_model = api.load(model_name)

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['tweet'])

In [ ]:
sequences = tokenizer.texts_to_sequences(df['tweet'])
padded_sequences = pad_sequences(sequences, maxlen=50, padding='post', truncating='post')

In [ ]:
Label_encoder = LabelEncoder()
Label_labels = Label_encoder.fit_transform(df['Label_cat'])
y1 = tf.keras.utils.to_categorical(Label_labels, num_classes=3)

In [ ]:
embedding_dim = 300
num_words = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.key_to_index.keys():
        embedding_matrix[i] = w2v_model[word]

# Define the model architecture
input_layer = Input(shape=(50,))
embedding_layer = Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False)(input_layer)
lstm_layer = LSTM(128, return_sequences=True)(embedding_layer)
lstm_layer = Dropout(0.5)(lstm_layer)
lstm_layer = LSTM(64)(lstm_layer)
lstm_layer = Dropout(0.5)(lstm_layer)
output_layer = Dense(3, activation='softmax')(lstm_layer)

model = Model(inputs=[input_layer], outputs=output_layer)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(padded_sequences, y1, batch_size=32, epochs=50, validation_split=0.2)


Epoch 1/50
504/504 [==============================] - 10s 11ms/step - loss: 0.9633 - accuracy: 0.5380 - val_loss: 1.0784 - val_accuracy: 0.3596
Epoch 2/50
504/504 [==============================] - 4s 9ms/step - loss: 0.8486 - accuracy: 0.6094 - val_loss: 1.0531 - val_accuracy: 0.4648
Epoch 3/50
504/504 [==============================] - 6s 12ms/step - loss: 0.8016 - accuracy: 0.6452 - val_loss: 1.0477 - val_accuracy: 0.4871
Epoch 4/50
504/504 [==============================] - 4s 9ms/step - loss: 0.7635 - accuracy: 0.6706 - val_loss: 0.9745 - val_accuracy: 0.5457
Epoch 5/50
504/504 [==============================] - 4s 9ms/step - loss: 0.7250 - accuracy: 0.6930 - val_loss: 0.9956 - val_accuracy: 0.5437
Epoch 6/50
504/504 [==============================] - 5s 10ms/step - loss: 0.6835 - accuracy: 0.7161 - val_loss: 1.0992 - val_accuracy: 0.5092
Epoch 7/50
504/504 [==============================] - 4s 9ms/step - loss: 0.6521 - accuracy: 0.7371 - val_loss: 1.0564 - val_accuracy: 0.5134
Ep

IndexError: ignored